# K2-18b gas dwarf models

In [ ]:
from atmodeller import Species, InteriorAtmosphere, Planet, earth_oceans_to_hydrogen_mass
from atmodeller import debug_logger
from atmodeller.solubility import get_solubility_models
from atmodeller.eos import get_eos_models
from atmodeller.thermodata import IronWustiteBuffer
from atmodeller.eos._holland_powell import H2O_cork_holland98
import logging
import numpy as np

np.random.seed(0)

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

Parameters for the simulations

In [2]:
number_of_realisations = 10000
surface_temperature = 2000.0  # Must be float
planet_mass = 5.15211e25
surface_radius = 1.6647e7
mantle_melt_fraction = 1.0

In [3]:
k218b = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

## Ideal gas no solubility

In [9]:
H2O_g = Species.create_gas("H2O_g")
H2_g = Species.create_gas("H2_g")
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g")
CO2_g = Species.create_gas("CO2_g")
CH4_g = Species.create_gas("CH4_g")
N2_g = Species.create_gas("N2_g")
NH3_g = Species.create_gas("NH3_g")

species = (H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g)

In [ ]:
interior_atmosphere_nosol = InteriorAtmosphere(species)

# Log uniform sampling
log10_number_oceans = np.random.uniform(0, 3.5, number_of_realisations)
log10_ch_ratios = np.random.uniform(-2, 1, number_of_realisations)
log10_nh_ratios = np.random.uniform(-4, -1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": h_kg * 10**log10_nh_ratios,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
initial_log_number_density[2] = 35.0
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_nosol.initialise_solve(
    planet=k218b,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_nosol = interior_atmosphere_nosol.solve()

# Quick look at the solution
# solution = output_nosol.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output_nosol.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
output_nosol.to_excel("k218b_ideal_nosol")

# Write the data to a pickle file with dataframes
output_nosol.to_pickle("k218b_ideal_nosol")

## Ideal gas with solubility

In [11]:
solubility_models = get_solubility_models()

In [12]:
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2_g", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2_g", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4_g", solubility=solubility_models["CH4_basalt_ardia13"])
N2_g = Species.create_gas("N2_g", solubility=solubility_models["N2_basalt_libourel03"])
NH3_g = Species.create_gas("NH3_g")

species = (H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g)

In [ ]:
interior_atmosphere_withsol = InteriorAtmosphere(species)

# Log uniform sampling
log10_number_oceans = np.random.uniform(0, 3.5, number_of_realisations)
log10_ch_ratios = np.random.uniform(-2, 1, number_of_realisations)
log10_nh_ratios = np.random.uniform(-4, -1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": h_kg * 10**log10_nh_ratios,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
initial_log_number_density[2] = 35.0
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_withsol.initialise_solve(
    planet=k218b,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_withsol = interior_atmosphere_withsol.solve()

# Quick look at the solution
# solution = output_withsol.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output_withsol.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
output_withsol.to_excel("k218b_ideal_withsol")

# Write the data to a pickle file with dataframes
output_withsol.to_pickle("k218b_ideal_withsol")

## Real gas no solubility

In [4]:
eos_models = get_eos_models()

In [5]:
H2O_g = Species.create_gas(
    "H2O_g",
    # activity=H2O_cork_holland98,
    # activity=eos_models["H2O_cork_holland98"],
)
H2_g = Species.create_gas("H2_g")  # , activity=eos_models["H2_shi92"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas(
    "CO_g",
    # activity=eos_models["CO_cork_cs_holland91"],
)
CO2_g = Species.create_gas(
    "CO2_g",
    # activity=eos_models["CO2_cork_holland98"],
)
CH4_g = Species.create_gas(
    "CH4_g",
    # activity=eos_models["CH4_cork_cs_holland91"],
)
N2_g = Species.create_gas(
    "N2_g",
    # activity=eos_models["N2_cork_cs_holland91"],
)
NH3_g = Species.create_gas("NH3_g")

species = (H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g)

In [ ]:
interior_atmosphere_real_nosol = InteriorAtmosphere(species)

# Original full
# Log uniform sampling
log10_number_oceans = np.random.uniform(0, 3.5, number_of_realisations)
log10_ch_ratios = np.random.uniform(-2, 1, number_of_realisations)
log10_nh_ratios = np.random.uniform(-4, -1, number_of_realisations)

fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)
# fO2_log10_shifts = np.random.uniform(-5, 0, number_of_realisations)
# fO2_log10_shifts = np.random.uniform(0, 5, number_of_realisations)

# Smaller sample for testing
# log10_number_oceans = 1
# log10_ch_ratios = 1
# log10_nh_ratios = 0.1
# fO2_log10_shifts = 0

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": h_kg * 10**log10_nh_ratios,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
# initial_log_number_density[2] = 35.0
# These guesses are taking the median values with log10_number_oceans max of unity and
# log10_nh_ratios fixed at 0.1
# initial_log_number_density = np.array([61, 60, 40, 57, 56, 48, 59, 52], dtype=np.float_)
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_real_nosol.initialise_solve(
    planet=k218b,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_real_nosol = interior_atmosphere_real_nosol.solve()

# Quick look at the solution
# solution = output_real_nosol.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output_real_nosol.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
output_real_nosol.to_excel("k218b_real_nosol")

# Write the data to a pickle file with dataframes
output_real_nosol.to_pickle("k218b_real_nosol")

## Real gas with solubility

In [9]:
eos_models = get_eos_models()
solubility_models = get_solubility_models()

In [ ]:
H2O_g = Species.create_gas(
    "H2O_g",
    activity=eos_models["H2O_cork_holland98"],
    solubility=solubility_models["H2O_basalt_dixon95"],
)
H2_g = Species.create_gas(
    "H2_g", activity=eos_models["H2_shi92"], solubility=solubility_models["H2_basalt_hirschmann12"]
)
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas(
    "CO_g",
    activity=eos_models["CO_cork_cs_holland91"],
    solubility=solubility_models["CO_basalt_yoshioka19"],
)
CO2_g = Species.create_gas(
    "CO2_g",
    activity=eos_models["CO2_cork_holland98"],
    solubility=solubility_models["CO2_basalt_dixon95"],
)
CH4_g = Species.create_gas(
    "CH4_g",
    activity=eos_models["CH4_cork_cs_holland91"],
    solubility=solubility_models["CH4_basalt_ardia13"],
)
N2_g = Species.create_gas(
    "N2_g",
    activity=eos_models["N2_cork_cs_holland91"],
    solubility=solubility_models["N2_basalt_libourel03"],
)
NH3_g = Species.create_gas("NH3_g")

species = (H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g)

In [ ]:
interior_atmosphere_real_withsol = InteriorAtmosphere(species)

# log10_number_oceans = np.random.uniform(0, 3.5, number_of_realisations)
log10_number_oceans = np.random.uniform(0, 1, number_of_realisations)
log10_ch_ratios = np.random.uniform(-2, 1, number_of_realisations)
# log10_nh_ratios = np.random.uniform(-4, -1, number_of_realisations)
log10_nh_ratios = 0.1
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
mass_constraints = {
    "H": h_kg,
    "C": h_kg * 10**log10_ch_ratios,
    "N": h_kg * 10**log10_nh_ratios,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
# In practice, I run a smaller model to get a sense of the average number densities, and then tweak
# these estimates accordingly.
initial_log_number_density = 55 * np.ones(len(species), dtype=np.float_)
# fO2 will benefit from a smaller initial estimate
initial_log_number_density[2] = 35.0
# Initial solution guess species stability
initial_log_stability = -135 * np.ones_like(initial_log_number_density)

# Precompile
interior_atmosphere_real_withsol.initialise_solve(
    planet=k218b,
    initial_log_number_density=initial_log_number_density,
    initial_log_stability=initial_log_stability,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)

output_real_withsol = interior_atmosphere_real_withsol.solve()

# Quick look at the solution
# solution = output_real_withsol.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output_real_withsol.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
output_real_withsol.to_excel("k218b_real_withsol")

# Write the data to a pickle file with dataframes
output_real_withsol.to_pickle("k218b_real_withsol")